In [1]:
cd /Users/jz-rolling/Desktop/OMEGA_GUI/notebooks/

/Users/jz-rolling/Desktop/OMEGA_GUI/notebooks


In [2]:
#import OMEGA2 as om
import tifffile
from skimage import filters,morphology
from momia2.segmentation import *
from momia2.utils import bandpass, correction,generic
import numpy as np, glob
import momia2 as mo, tifffile

In [3]:
def xydrift_correction(target_img, shift, max_drift=1000):
    
    if max(np.abs(shift)) <= max_drift:
        return shift_image(target_img, np.array(shift))
    else:
        return target_img
    
def get_xydrift(ref_img, target_img):
    from skimage import registration
    shift, error, _diff = registration.phase_cross_correlation(ref_img, target_img, upsample_factor=10)
    return shift

def shift_image(img, shift):
    from scipy import ndimage as ndi
    """
    correct xy drift between phase contrast image and fluorescent image(s)
    :param img: input image
    :param shift: subpixel xy drift
    :return: drift corrected image
    """
    offset_image = ndi.fourier_shift(np.fft.fftn(img), shift)
    offset_image = np.fft.ifftn(offset_image)
    offset_image = np.round(offset_image.real)
    offset_image[offset_image <= 0] = 1
    # rescaled to avoid int16 overflow
    offset_image[offset_image>=65530] = 65530
    return offset_image.astype(np.uint16)


def correct_tl(pos,softread_data,n_channels=2,ref_channel=1):
    pos_data = {i:[] for i in range(n_channels)}
    output_data = []
    for c in range(n_channels):
        for t in range(softread_data['n_timepoints']):
            pos_data[c].append(mo.io.get_slice_by_index(softread_data,channel=c,position=pos,time=t))
        pos_data[c]=np.array(pos_data[c])
    
    drift_corrected = {i:np.zeros(pos_data[i].shape) for i in range(n_channels)}
    last_drift = np.array([0,0])
    max_drift = 0
    print(len(pos_data[0]))
    for t in range(softread_data['n_timepoints']):
        t_frame = []
        if t == 0:
            for c in range(n_channels):
                t_frame.append(pos_data[c][t])
        else:
            last_drift = last_drift + get_xydrift(pos_data[ref_channel][t-1],pos_data[ref_channel][t])
            max_drift = max(abs(np.max(last_drift)),max_drift)
            for c in range(n_channels):
                t_frame.append(xydrift_correction(pos_data[c][t],last_drift))
        output_data.append(np.array(t_frame))
    output_data = np.array(output_data)
    max_drift = int(round(max_drift))
    cropped = output_data[:,:,max_drift:-max_drift,max_drift:-max_drift]
    return cropped

def correct_tl_simp(pos,timearray,n_channels=1,ref_channel=0):
    pos_data = {i:[] for i in range(n_channels)}
    output_data = []
    for c in range(n_channels):
        for t in range(softread_data['n_timepoints']):
            pos_data[c].append(mo.io.get_slice_by_index(softread_data,channel=c,position=pos,time=t))
        pos_data[c]=np.array(pos_data[c])
    
    drift_corrected = {i:np.zeros(pos_data[i].shape) for i in range(n_channels)}
    last_drift = np.array([0,0])
    max_drift = 0
    print(len(pos_data[0]))
    for t in range(softread_data['n_timepoints']):
        t_frame = []
        if t == 0:
            for c in range(n_channels):
                t_frame.append(pos_data[c][t])
        else:
            last_drift = last_drift + get_xydrift(pos_data[ref_channel][t-1],pos_data[ref_channel][t])
            max_drift = max(abs(np.max(last_drift)),max_drift)
            for c in range(n_channels):
                t_frame.append(xydrift_correction(pos_data[c][t],last_drift))
        output_data.append(np.array(t_frame))
    output_data = np.array(output_data)
    max_drift = int(round(max_drift))
    cropped = output_data[:,:,max_drift:-max_drift,max_drift:-max_drift]
    return cropped

In [6]:
soft=mo.io.softread_file('/Volumes/JZSSD_temp/202212_Harry_plate_analysis/rpoA_timelapse_001.nd2')
for i in range(8):
    drift_corrected=correct_tl(i,soft)
    tifffile.imwrite('/Volumes/JZSSD_temp/202212_Harry_plate_analysis/rpoA_timelapse_pos_{}.tif'.format(i), 
                     drift_corrected,imagej=True,
                     metadata={'axes': 'TCYX'})

49
49
49
49
49
49
49
49
